# BTK Akademi Datathon 2024

## Import Libraries

In [ ]:
# Necessary libraries
import re
import optuna
import numpy as np 
import pandas as pd 
import seaborn as sns  
import catboost as cb
from datetime import datetime
from unidecode import unidecode
import matplotlib.pyplot as plt 
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder

import warnings 
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Examine the Dataset

In [ ]:
# Datasets
train_data = pd.read_csv("/kaggle/input/datathon-2024/train.csv")
test_data = pd.read_csv("/kaggle/input/datathon-2024/test_x.csv")
samp_sub = pd.read_csv("/kaggle/input/datathon-2024/sample_submission.csv")
il_ilce_list = pd.read_csv("/kaggle/input/trkiye-il-ile-listesi/il_ilce.csv")
uni_list = pd.read_csv("/kaggle/input/turkiyedeki-universiteler/turkiye_universiteleri.csv", sep = ';')

df_train = train_data.copy()
df_test = test_data.copy()

#Drop id from train set
df_train = df_train.drop(columns = 'id', axis = 1)

In [ ]:
# Properties of Dataset 
def analyze_dataset(df, name):
    print(f"\n--- {name} Veri Seti Analizi ---")
    print(f"Boyut: {df.shape}")
    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print("\nVeri Tipleri:")
    print(df.dtypes)
    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print("\nEksik Veriler:")
    print((df.isnull().sum() / len(df) * 100).round(2))
    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    print("\nBetimsel İstatistikler:")
    print(df.describe())
    
analyze_dataset(df_train, "Eğitim")
analyze_dataset(df_test, "Test")

In [ ]:
# Variables that are 100% empty in the test set were dropped.
empty_columns = ["Burslu ise Burs Yuzdesi", "Daha Once Baska Bir Universiteden Mezun Olmus", "Lise Adi Diger", "Lise Bolum Diger", 
                 "Uye Oldugunuz Kulubun Ismi", "Stk Projesine Katildiniz Mi?", "Ingilizce Seviyeniz?", "Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite"]

df_train = df_train.drop(columns = empty_columns, axis = 1)
df_test = df_test.drop(columns = empty_columns, axis = 1)

## Variable Adjustments

In [ ]:
#-------------------------------------------------------------------------------------------------------------#

# Adjusting column names
def clean_column_name(name):
    return ''.join(c if c.isalnum() or c == '_' else '_' for c in name)
df_train.columns = [clean_column_name(col) for col in df_train.columns]
df_test.columns = [clean_column_name(col) for col in df_test.columns]

#-------------------------------------------------------------------------------------------------------------#

# Evaluation score
df_train = df_train[df_train['Degerlendirme_Puani'] <= 100] # Describe alınca max değer 102 gözüküyordu.

#-------------------------------------------------------------------------------------------------------------#
# Misspelling 

df_train['Cinsiyet'] = df_train['Cinsiyet'].replace({'ERKEK': 'Erkek'})

#-------------------------------------------------------------------------------------------------------------#

# Extract the year
df_train['Dogum_Tarihi'] = pd.to_datetime(df_train['Dogum_Tarihi'], errors='coerce', dayfirst=True)
df_train['Dogum_Tarihi'] = df_train['Dogum_Tarihi'].fillna(pd.Timestamp('2000-01-01'))
df_train['Dogum_Tarihi'] = df_train['Dogum_Tarihi'].dt.year

df_test['Dogum_Tarihi'] = pd.to_datetime(df_test['Dogum_Tarihi'], errors='coerce', dayfirst=True)
df_test['Dogum_Tarihi'] = df_test['Dogum_Tarihi'].fillna(pd.Timestamp('2000-01-01'))
df_test['Dogum_Tarihi'] = df_test['Dogum_Tarihi'].dt.year

#-------------------------------------------------------------------------------------------------------------#

# Extract the city names
def create_city_mapping(il_ilce_list):
    """
    Creates a mapping of all possible spellings to correct city names.
    
    :param il_ilce_list: DataFrame containing 'il' and 'ilce' columns
    :return: Dictionary mapping all spellings to correct city names
    """
    mapping = {}
    for _, row in il_ilce_list.iterrows():
        il = row['il']
        ilce = row['ilce']
        
        # Add variations of city name
        mapping[unidecode(il.lower())] = il
        mapping[unidecode(ilce.lower())] = il
        
        # Add common misspellings
        mapping[unidecode(il.lower().replace('ı', 'i'))] = il
        mapping[unidecode(ilce.lower().replace('ı', 'i'))] = il
        
        # Add variations without spaces
        mapping[unidecode(il.lower().replace(' ', ''))] = il
        mapping[unidecode(ilce.lower().replace(' ', ''))] = il
        
    return mapping

def correct_city_names(df, city_column, il_ilce_list):
    """
    Corrects and standardizes city names in a DataFrame using il_ilce_list.
    
    :param df: DataFrame containing the city column to be corrected
    :param city_column: Name of the column containing city names
    :param il_ilce_list: DataFrame containing 'il' and 'ilce' columns
    :return: Series with corrected city names
    """
    city_mapping = create_city_mapping(il_ilce_list)
    
    def clean_city_name(name):
        if pd.isna(name):
            return ""
        if not isinstance(name, str):
            return str(name)
        # Remove parentheses and their contents
        name = re.sub(r'\([^)]*\)', '', name)
        # Remove digits and special characters
        name = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', name)
        # Remove leading/trailing whitespace and convert to title case
        return name.strip().title()
    
    def find_match(name):
        cleaned_name = clean_city_name(name)
        if not cleaned_name:
            return np.nan
        
        normalized_name = unidecode(cleaned_name.lower())
        
        # Check for exact matches first
        if normalized_name in city_mapping:
            return city_mapping[normalized_name]
        
        # Check individual words
        words = normalized_name.split()
        for word in words:
            if word in city_mapping:
                return city_mapping[word]
        
        # If still no match, try partial matching
        for key in city_mapping:
            if key in normalized_name or normalized_name in key:
                return city_mapping[key]
        
        # If no match found, return the original cleaned name
        return cleaned_name
    
    # Apply the correction and return a Series
    return df[city_column].apply(find_match)

# Usage
df_train['Dogum_Yeri'] = correct_city_names(df_train, 'Dogum_Yeri', il_ilce_list)
df_test['Dogum_Yeri'] = correct_city_names(df_test, 'Dogum_Yeri', il_ilce_list)

replace_list = [
    'Grossumstadtalmanya', 'State College Abd', 'Melbourneavustralia', 'Abington  Usa', 'Kazakistan',
    'İsviçrerheinfelden', 'Nottingham İngi̇ltere', 'Alabamaabd', 'Duisburg', 'Wuppertal Almanya',
    'Wuppertal Almanya', 'Bratosh Shkoder Arnavutluk', 'Leventist', 'Tehran  Iran', 'Rusya', 'Lefkoşa',
    'Charsaddapakistan', 'Bulgaristan', 'Kazakistan Türk Vatandaşıyım', 'Ukraynasevastopol', 'Mekkesarabistan',
    'Mekke', 'Bristol', 'Trablus Libya', 'Suudi Arabistan', 'Viyana', 'Berlin', 'Demokratik Kongo Cumhuriyeti Kinshasa',
    'Tervelbulgaristan', 'Gleize  Fransa', 'Almanyakempten', 'Macoo', 'Gurcistan  Bolnisi', 'Stockholm  İsveç',
    'Schorndorfalmanya', 'Kahire', 'Bakü', 'Edehollanda', 'Khta', 'Kırgızistanoş', 'Hollanda', 'Ci̇ddesj',
    'Köln  Almanya', 'Kahiremısır', 'Bolton  Ingiltere', 'Tacikistan', 'Almanya', 'Trablus  Libya'
]

#-------------------------------------------------------------------------------------------------------------#

# Replace these values with "Yurt Dışı"
df_train['Dogum_Yeri'] = df_train['Dogum_Yeri'].replace(replace_list, 'Yurt Dışı')

#-------------------------------------------------------------------------------------------------------------#

# Replace these values with real names
df_train['Dogum_Yeri'] = df_train['Dogum_Yeri'].replace({'Kopnya': 'Konya','Kmaras': 'Kahramanmaraş','Kmaraş': 'Kahramanmaraş','Hakkri': 'Hakkâri',
                                                         'Çekirge': 'Osmangazi','Erurum': 'Erzurum','Glresun': 'Giresun','Üskidar': 'İstanbul',
                                                         'Eskişeir': 'Eskişehir','Sansun': 'Samsun','Emnönü': 'İstanbul','Cumhuriyet Mahallesilale Sokakerdil Apartmanıno Daire': 'Manisa',
                                                         'Vezi̇rkörü': 'Samsun','Gantep': 'Gaziantep','Sultanbeliist': 'İstanbul','Taksi̇m': 'İstanbul',
                                                         'Mrsin': 'Mersin','Gopaşa': 'İstanbul','Siivas': 'Sivas','Altıntş': 'Antalya','Doğubeyazıt': 'Ağrı',
                                                         'Şanlıuırfa': 'Şanlıurfa','Kktcmagosa': 'Kktc','Osmangazi':'Bursa', 'Şanluırfa':'Şanlıurfa','Türkiye': np.nan,'Diğer':np.nan,'Diger':np.nan})

#-------------------------------------------------------------------------------------------------------------#

# Extract the city names
df_train['Ikametgah_Sehri'] = correct_city_names(df_train, 'Ikametgah_Sehri', il_ilce_list)
df_test['Ikametgah_Sehri'] = correct_city_names(df_test, 'Ikametgah_Sehri', il_ilce_list)

df_train['Ikametgah_Sehri'] = df_train['Ikametgah_Sehri'].replace({'Girne': 'Kktc','Gazi̇mağusa': 'Kktc','Gazimağusa': 'Kktc',
                                                                  'Kmaraş': 'Kahramanmaraş','Lefkoşa': 'Kktc','Lefke': 'Kktc',
                                                                  'Diger': np.nan,'Hakkri': 'Hakkâri','Diğer': np.nan,
                                                                  'Hamburg': 'Yurt Dışı','Kırgızistanbişkek': 'Yurt Dışı',
                                                                  'Nürnberg': 'Yurt Dışı','Gotham': 'Yurt Dışı',
                                                                  'Gothamcity': 'Yurt Dışı','Oslo':'Yurt Dışı',
                                                                  'Openairequestfailedtimeout' : np.nan,
                                                                  'Bakü': 'Yurt Dışı','Şanluırfa': 'Şanlıurfa',
                                                                  'Osmangazi': 'Bursa'})

#-------------------------------------------------------------------------------------------------------------#

# University type misspelling
df_train['Universite_Turu'] = df_train['Universite_Turu'].replace({'DEVLET': 'Devlet', 'ÖZEL': 'Özel'})

#-------------------------------------------------------------------------------------------------------------#

# "Is he/she receiving a scholarship?" misspelling
df_train['Burs_Aliyor_mu_'] = df_train['Burs_Aliyor_mu_'].replace({'hayır': 'Hayır', 'evet': 'Evet', 'EVET': 'Evet'})

#-------------------------------------------------------------------------------------------------------------#

# Department
top_50_categories = df_train["Bölüm"].value_counts().head(50).index
df_train["Bölüm"] = df_train["Bölüm"].apply(lambda x: x if x in top_50_categories else "Other")
df_test["Bölüm"] = df_test["Bölüm"].apply(lambda x: x if x in top_50_categories else "Other")

#-------------------------------------------------------------------------------------------------------------#

# Uni year misspelling
df_train['Universite_Kacinci_Sinif'] = df_train['Universite_Kacinci_Sinif'].replace({'hazırlık': 'Hazırlık', '0': 'Hazırlık', 'Tez': '4'})

#-------------------------------------------------------------------------------------------------------------#

# Uni Gano
df_train['Universite_Not_Ortalamasi'] = df_train['Universite_Not_Ortalamasi'].replace({'3.50-3': '3.00 - 3.49', '3.00-2.50': '2.50 - 2.99',
                                                                                       '2.50 ve altı': '1.80 - 2.49','3.00 - 4.00': '3.00 - 3.49',
                                                                                       '3.50 - 4.00': '3.50 - 4.00','3.00 - 3.50': '3.00 - 3.49',
                                                                                       '2.50 -3.00': '2.50 - 2.99','4-3.5': '3.50 - 4.00',
                                                                                       '2.50 - 3.00': '2.50 - 2.99','2.00 - 2.50': '1.80 - 2.49',
                                                                                       '1.00 - 2.50': '1.80 - 2.49','4.0-3.5': '3.50 - 4.00',
                                                                                       '3.00 - 3.49': '3.00 - 3.49','2.50 - 2.99': '2.50 - 2.99',
                                                                                       '1.80 - 2.49': '1.80 - 2.49','0 - 1.79': '0 - 1.79',
                                                                                       'Ortalama bulunmuyor': np.nan,'ORTALAMA BULUNMUYOR': np.nan,
                                                                                       'Not ortalaması yok': np.nan})

#-------------------------------------------------------------------------------------------------------------#

# High school misspelling
df_train['Lise_Sehir'] = correct_city_names(df_train, 'Lise_Sehir', il_ilce_list)
df_test['Lise_Sehir'] = correct_city_names(df_test, 'Lise_Sehir', il_ilce_list)

df_train['Lise_Sehir'] = df_train['Lise_Sehir'].replace({'Corly': 'Yurt Dışı', 'Diğer': np.nan,
                                                        'Yeşi̇lköy': 'İstanbul', 'Ferizajkosova': 'Yurt Dışı',
                                                        'Viranşehie': 'Yurt Dışı', 'Zakatala': 'Yurt Dışı',
                                                        'Isranbul': 'İstanbul','Abd Missouri': 'Yurt Dışı',
                                                        'Tursunzade':'Yurt Dışı', 'Peshawar': 'Yurt Dışı',
                                                        'Kahramanmaas': 'Kahramanmaraş', 'Diyarakır' :'Diyarbakır',
                                                        'Bulut': np.nan, 'Hakkri': 'Hakkari',
                                                        'Kahramanmraş': 'Kahramanmaraş', 'İsyanbul': 'İstanbul',
                                                        'Mrsin': 'Mersin', 'Ankata': 'Ankara',
                                                        'Ateş': np.nan, 'Bakıoğlu': np.nan,
                                                        'Kinshasa':'Yurt Dışı', 'Montezumanew Mexico': 'Yurt Dışı',
                                                        'Yılmaz':np.nan, 'Kürkcü':np.nan,
                                                        'Sumgayıt':'Yurt Dışı', 'Kmaraş': 'Kahramanmaraş',
                                                        'Türkiye': np.nan, 'Karabbük':'Karabük',
                                                        'İstanbuk': 'İstanbul', 'Malatua':'Malatya',
                                                        'Baku':'Yurt Dışı', 'Eskişeir': 'Eskişehir',
                                                        'Antaya': 'Antalya', 'İstabul':'İstanbul',
                                                        'Gebe': 'Gaziantep', 'Shkoder':'Yurt Dışı',
                                                        'Bakü':'Yurt Dışı', 'Atyrau':'Yurt Dışı',
                                                        'Istabul':'İstanbul', 'Almanya':'Yurt Dışı',
                                                        'Diger': np.nan, 'Tranzon':'Trabzon',
                                                        'Asdasd': np.nan, 'Gantep': 'Gaziantep',
                                                        'Lefkoşakktc': 'Kktc', 'Erzurun':'Erzurum','Hakkari':'Hakkâri',
                                                        'Lefkoşa':'Kktc','Kocaei̇':'Kocaeli', 'Kolej': 'Ankara'})

#-------------------------------------------------------------------------------------------------------------#

# High School type misspelling
df_train['Lise_Turu'] = df_train['Lise_Turu'].replace({'Anadolu lisesi': 'Devlet',
                                                       'Meslek lisesi': 'Devlet',
                                                       'Fen lisesi': 'Devlet',
                                                       'Özel lisesi': 'Özel',
                                                       'Düz lise': 'Devlet',
                                                       'İmam Hatip Lisesi': 'Devlet',
                                                       'Meslek Lisesi': 'Devlet',
                                                       'Fen Lisesi': 'Devlet',
                                                       'Düz Lise': 'Devlet',
                                                       'Anadolu Lisesi': 'Devlet',
                                                       'Özel Lise': 'Özel',
                                                       'Özel Lisesi': 'Özel',
                                                       'Meslek': 'Devlet',
                                                       'Diğer': 'Devlet'})

# Drop High School Department
df_train = df_train.drop(columns = ['Lise_Bolumu'], axis = 1)
df_test = df_test.drop(columns = ['Lise_Bolumu'], axis = 1)

#-------------------------------------------------------------------------------------------------------------#

# High School GDP misspelling
df_train['Lise_Mezuniyet_Notu'] = df_train['Lise_Mezuniyet_Notu'].replace({'3.50-3': '75 - 100',
                                                                           '3.00 - 4.00': '75 - 100',
                                                                           '3.00-2.50': '50 - 74',
                                                                           '2.50 ve altı': '25 - 49',
                                                                           '4.00-3.50': '75 - 100',
                                                                           '3.50-3.00': '75 - 100',
                                                                           '69-55': '50 - 74',
                                                                           '100-85': '75 - 100',
                                                                           '84-70': '75 - 100',
                                                                           '54-45': '50 - 74',
                                                                           '44-0': '25 - 49',
                                                                           '75 - 100': '75 - 100',
                                                                           '50 - 75': '50 - 74',
                                                                           'Not ortalaması yok': np.nan,
                                                                           '25 - 50': '25 - 49',
                                                                           '0 - 25': '25 - 49',
                                                                           '50 - 74': '50 - 74',
                                                                           '25 - 49': '25 - 49',
                                                                           '0 - 24': '25 - 49'})

#-------------------------------------------------------------------------------------------------------------#

# Burs_Aldigi_Baska_Kurum misspelling
df_train['Burs_Aldigi_Baska_Kurum'] = df_train['Burs_Aldigi_Baska_Kurum'].replace({'-': np.nan,
                                                                                  'Kyk': 'KYK',
                                                                                  'kyk': 'KYK',
                                                                                  'Kredi ve Yurtlar Kurumu': 'KYK',
                                                                                  'Kyk bursu': 'KYK',
                                                                                  'Devlet': 'KYK',
                                                                                  'Kredi Yurtlar Kurumu': 'KYK',
                                                                                  'Kredi yurtlar kurumu': 'KYK',
                                                                                  'Kredi ve yurtlar kurumu': 'KYK',
                                                                                  'KYK bursu': 'KYK',
                                                                                  'K': 'KYK',
                                                                                  'Devlet bursu': 'KYK',
                                                                                  'kredi yurtlar kurumu /kredi alıyorum': 'KYK',
                                                                                  'KYK Yök bursu': 'KYK',
                                                                                  'KYK Geri odemeli kredi aliyorum': 'KYK',
                                                                                  'kredi yurtlar kurumu': 'KYK',
                                                                                  'Kredi yurtlar kurumu ögrenim bursu': 'KYK'})

df_test['Burs_Aldigi_Baska_Kurum'] = df_test['Burs_Aldigi_Baska_Kurum'].replace({'-': np.nan,
                                                                                  'Kyk': 'KYK',
                                                                                  'kyk': 'KYK',
                                                                                  'Kredi ve Yurtlar Kurumu': 'KYK',
                                                                                  'Kyk bursu': 'KYK',
                                                                                  'Devlet': 'KYK',
                                                                                  'Kredi Yurtlar Kurumu': 'KYK',
                                                                                  'Kredi yurtlar kurumu': 'KYK',
                                                                                  'Kredi ve yurtlar kurumu': 'KYK',
                                                                                  'KYK bursu': 'KYK',
                                                                                  'K': 'KYK',
                                                                                  'Devlet bursu': 'KYK',
                                                                                  'kredi yurtlar kurumu /kredi alıyorum': 'KYK',
                                                                                  'KYK Yök bursu': 'KYK',
                                                                                  'KYK Geri odemeli kredi aliyorum': 'KYK',
                                                                                  'kredi yurtlar kurumu': 'KYK',
                                                                                  'Kredi yurtlar kurumu ögrenim bursu' : 'KYK'})

#-------------------------------------------------------------------------------------------------------------#

# Update Scholarship Information and Apply Threshold for Value Counts
df_train.loc[(df_train['Baska_Bir_Kurumdan_Burs_Aliyor_mu_'] == 'Hayır') & (df_train['Burs_Aldigi_Baska_Kurum'].isna()), 'Burs_Aldigi_Baska_Kurum'] = 'Burs almıyor'
df_test.loc[(df_test['Baska_Bir_Kurumdan_Burs_Aliyor_mu_'] == 'Hayır') & (df_test['Burs_Aldigi_Baska_Kurum'].isna()), 'Burs_Aldigi_Baska_Kurum'] = 'Burs almıyor'

frequency_test = df_test['Burs_Aldigi_Baska_Kurum'].value_counts()
frequency_train = df_train['Burs_Aldigi_Baska_Kurum'].value_counts(dropna=True)

threshold = 100

df_test['Burs_Aldigi_Baska_Kurum'] = df_test['Burs_Aldigi_Baska_Kurum'].apply(lambda x: x if frequency_test[x] >= threshold else 'Diğer')
df_train['Burs_Aldigi_Baska_Kurum'] = df_train['Burs_Aldigi_Baska_Kurum'].apply(lambda x: x if pd.isna(x) or frequency_train.get(x, 0) >= threshold else 'Diğer')

#-------------------------------------------------------------------------------------------------------------#

# Data Cleaning and Classification for Scholarship Amounts from Other Institutions
df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'] = df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'].replace({'-': np.nan})
df_test['Baska_Kurumdan_Aldigi_Burs_Miktari'] = df_test['Baska_Kurumdan_Aldigi_Burs_Miktari'].replace({'-': np.nan})

df_train.loc[(df_train['Baska_Bir_Kurumdan_Burs_Aliyor_mu_'] == 'Hayır') & (df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'].isna()), 'Baska_Kurumdan_Aldigi_Burs_Miktari'] = '0'
df_test.loc[(df_test['Baska_Bir_Kurumdan_Burs_Aliyor_mu_'] == 'Hayır') & (df_test['Baska_Kurumdan_Aldigi_Burs_Miktari'].isna()), 'Baska_Kurumdan_Aldigi_Burs_Miktari'] = '0'

def extract_numbers(text):
    if isinstance(text, str):
        numbers = re.findall(r'\d+', text)
        return int(max(numbers, key=int)) if numbers else 0
    return 0

df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'] = df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'].apply(extract_numbers)

def classify_scholarship(amount):
    if amount == 0:
        return '0'
    elif 0 < amount < 500:
        return '0 - 499 ₺'
    elif 500 <= amount < 1000:
        return '500₺ - 999₺'
    else:
        return '1000₺ ve üstü'

df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'] = df_train['Baska_Kurumdan_Aldigi_Burs_Miktari'].apply(classify_scholarship)

#-------------------------------------------------------------------------------------------------------------#

# Standardize Education Status for Mothers in the Training Dataset
df_train['Anne_Egitim_Durumu'] = df_train['Anne_Egitim_Durumu'].replace({'İlkokul Mezunu': 'İlkokul',
                                                                         'Eğitim Yok': 'Eğitimi yok',
                                                                         'Ortaokul Mezunu': 'Ortaokul',
                                                                         'Yüksek Lisans / Doktora': 'Doktora',
                                                                         'Üniversite Mezunu': 'Üniversite',
                                                                         'Lise Mezunu': 'Lise',
                                                                         'Yüksek Lisans / Doktara': 'Doktora',
                                                                         'İLKOKUL MEZUNU': 'İlkokul',
                                                                         'LİSE': 'Lise',
                                                                         'EĞİTİM YOK': 'Eğitimi yok',
                                                                         'ÜNİVERSİTE': 'Üniversite',
                                                                         'ORTAOKUL MEZUNU': 'Ortaokul',
                                                                         'DOKTORA': 'Doktora',
                                                                         'YÜKSEK LİSANS': 'Yüksek Lisans'})

#-------------------------------------------------------------------------------------------------------------#

# Clean and Standardize Mother's Employment Sector and Status in Training and Test Datasets
df_train['Anne_Sektor'] = df_train['Anne_Sektor'].replace({'KAMU': 'Kamu',
                                                           'ÖZEL SEKTÖR': 'Özel Sektör',
                                                           'DİĞER': 'Diğer',
                                                           '-': np.nan})

df_test['Anne_Sektor'] = df_test['Anne_Sektor'].replace({'-': np.nan})

df_train.loc[df_train['Anne_Calisma_Durumu'].isin(['Hayır', 'Emekli']), 'Anne_Sektor'] = 0
df_test.loc[df_test['Anne_Calisma_Durumu'] == 'Hayır', 'Anne_Sektor'] = 0

df_train['Anne_Calisma_Durumu'] = df_train['Anne_Calisma_Durumu'].replace({'Emekli': 'Hayır'})

# Standardize Father's Education Status in the Training Dataset
df_train['Baba_Egitim_Durumu'] = df_train['Baba_Egitim_Durumu'].replace({'Yüksek Lisans / Doktora': 'Doktora',
                                                                         'İlkokul Mezunu': 'İlkokul',
                                                                         'Ortaokul Mezunu': 'Ortaokul',
                                                                         'Eğitim Yok': 'Eğitimi yok',
                                                                         '0': 'Eğitimi yok',
                                                                         'Üniversite Mezunu': 'Üniversite',
                                                                         'Lise Mezunu': 'Lise',
                                                                         'Yüksek Lisans / Doktara': 'Doktora',
                                                                         'İLKOKUL MEZUNU': 'İlkokul',
                                                                         'ÜNİVERSİTE': 'Üniversite',
                                                                         'EĞİTİM YOK': 'Eğitimi yok',
                                                                         'ORTAOKUL MEZUNU': 'Ortaokul',
                                                                         'LİSE': 'Lise',
                                                                         'YÜKSEK LİSANS': 'Yüksek Lisans',
                                                                         'DOKTORA': 'Doktora'})

#-------------------------------------------------------------------------------------------------------------#

# Clean and Standardize Father's Employment Sector and Status in Training and Test Datasets
df_train['Baba_Sektor'] = df_train['Baba_Sektor'].replace({'KAMU': 'Kamu',
                                                           'ÖZEL SEKTÖR': 'Özel Sektör',
                                                           'DİĞER': 'Diğer',
                                                           '-': np.nan})

df_test['Baba_Sektor'] = df_test['Baba_Sektor'].replace({'-': np.nan})

df_train.loc[df_train['Baba_Calisma_Durumu'].isin(['Hayır', 'Emekli']), 'Baba_Sektor'] = '0'
df_test.loc[df_test['Baba_Calisma_Durumu'] == 'Hayır', 'Baba_Sektor'] = '0'

df_train['Baba_Calisma_Durumu'] = df_train['Baba_Calisma_Durumu'].replace({'Emekli': 'Hayır'})

#-------------------------------------------------------------------------------------------------------------#

# Handle Specific Case for Sibling Count in the Training Datase
df_train.loc[df_train['Kardes_Sayisi'] == 'Kardeş Sayısı 1 Ek Bilgi Aile Hk. Anne Vefat', 'Kardes_Sayisi'] = np.nan

#-------------------------------------------------------------------------------------------------------------#

# Clean and Standardize Sports Role Information in Training and Test Datasets
df_train['Spor_Dalindaki_Rolunuz_Nedir_'] = df_train['Spor_Dalindaki_Rolunuz_Nedir_'].replace({'Kaptan': 'Lider/Kaptan',
                                                                                              'KAPTAN / LİDER': 'Lider/Kaptan',
                                                                                              'DİĞER': 'Diğer',
                                                                                              'Bireysel': 'Bireysel Spor',
                                                                                              '-': np.nan})

df_test['Spor_Dalindaki_Rolunuz_Nedir_'] = df_test['Spor_Dalindaki_Rolunuz_Nedir_'].replace({'-': np.nan})

df_train.loc[df_train['Profesyonel_Bir_Spor_Daliyla_Mesgul_musunuz_'] == 'Hayır', 'Spor_Dalindaki_Rolunuz_Nedir_'] = '0'
df_test.loc[df_test['Profesyonel_Bir_Spor_Daliyla_Mesgul_musunuz_'] == 'Hayır', 'Spor_Dalindaki_Rolunuz_Nedir_'] = '0'

#-------------------------------------------------------------------------------------------------------------#

# Drop Columns from Training and Test Datasets
df_train = df_train.drop(columns = ["Hangi_STK_nin_Uyesisiniz_"], axis = 1)
df_test = df_test.drop(columns = ["Hangi_STK_nin_Uyesisiniz_"], axis = 1)

#-------------------------------------------------------------------------------------------------------------#

# Drop Columns from Training and Test Datasets
df_train = df_train.drop(columns = ["Girisimcilikle_Ilgili_Deneyiminizi_Aciklayabilir_misiniz_"], axis = 1)
df_test = df_test.drop(columns = ["Girisimcilikle_Ilgili_Deneyiminizi_Aciklayabilir_misiniz_"], axis = 1)

## Handling Missing Values

In [ ]:
#-------------------------------------------------------------------------------------------------------------#

# Filling missing values with the mode
columns_to_fill_mode = ['Cinsiyet', 'Universite_Turu', 'Universite_Kacinci_Sinif', 'Universite_Not_Ortalamasi',
                       'Lise_Turu', 'Lise_Mezuniyet_Notu', 'Baska_Bir_Kurumdan_Burs_Aliyor_mu_', 'Dogum_Yeri',
                       'Lise_Sehir', 'Ikametgah_Sehri', 'Anne_Egitim_Durumu', 'Anne_Egitim_Durumu', 'Anne_Calisma_Durumu',
                       'Anne_Sektor', 'Baba_Egitim_Durumu', 'Baba_Calisma_Durumu', 'Baba_Sektor', 'Girisimcilik_Kulupleri_Tarzi_Bir_Kulube_Uye_misiniz_',
                       'Profesyonel_Bir_Spor_Daliyla_Mesgul_musunuz_', 'Spor_Dalindaki_Rolunuz_Nedir_', 'Aktif_olarak_bir_STK_üyesi_misiniz_',
                       'Girisimcilikle_Ilgili_Deneyiminiz_Var_Mi_']

for column in columns_to_fill_mode:
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)
    
#-------------------------------------------------------------------------------------------------------------#

# Filling missing values in the 'Birth Year' variable with the median birth year based on the application year
df_train['Dogum_Tarihi'] = pd.to_numeric(df_train['Dogum_Tarihi'], errors='coerce')
df_train['Dogum_Tarihi'] = df_train.groupby('Basvuru_Yili')['Dogum_Tarihi'].transform(lambda x: x.fillna(x.median()))
df_train['Dogum_Tarihi'] = df_train['Dogum_Tarihi'].astype(int)

#-------------------------------------------------------------------------------------------------------------#

# Filling missing values in the 'Number of Siblings' variable with the median number of siblings based on the place of birth
df_train['Kardes_Sayisi'] = pd.to_numeric(df_train['Kardes_Sayisi'], errors='coerce')
df_train['Kardes_Sayisi'] = df_train.groupby('Dogum_Yeri')['Kardes_Sayisi'].transform(lambda x: x.fillna(x.median()))
df_train['Kardes_Sayisi'] = df_train['Kardes_Sayisi'].astype(int)

#-------------------------------------------------------------------------------------------------------------#

# Encoding the application year as a binary feature for the year 2016
df_train['Basvuru_Yili_Weighted'] = df_train['Basvuru_Yili'].apply(lambda x: 1 if x == 2016 else 0)
df_test['Basvuru_Yili_Weighted'] = 0

#-------------------------------------------------------------------------------------------------------------#

# Filling missing values in the 'Burs_Aldigi_Baska_Kurum' column with the KYK
df_train['Burs_Aldigi_Baska_Kurum'].fillna('KYK', inplace = True)

#-------------------------------------------------------------------------------------------------------------#

# Filling missing values in the 'Ingilizce_Biliyor_musunuz_' column using the mode for each 'Lise_Turu' group
df_train['Ingilizce_Biliyor_musunuz_'] = df_train.groupby('Lise_Turu')['Ingilizce_Biliyor_musunuz_'].transform(lambda x: x.fillna(x.mode().iloc[0]))

## Encoding and Adjustments

In [ ]:
#-------------------------------------------------------------------------------------------------------------#

# One hot encoding
one_hot_encode_cols = ['Universite_Turu', 'Burs_Aliyor_mu_', 'Lise_Turu', 'Baska_Bir_Kurumdan_Burs_Aliyor_mu_',
                       'Burs_Aldigi_Baska_Kurum', 'Anne_Calisma_Durumu','Anne_Sektor', 'Baba_Calisma_Durumu',
                       'Baba_Sektor', 'Girisimcilik_Kulupleri_Tarzi_Bir_Kulube_Uye_misiniz_','Profesyonel_Bir_Spor_Daliyla_Mesgul_musunuz_',
                       'Spor_Dalindaki_Rolunuz_Nedir_', 'Aktif_olarak_bir_STK_üyesi_misiniz_','Girisimcilikle_Ilgili_Deneyiminiz_Var_Mi_', 'Ingilizce_Biliyor_musunuz_']

df_train = pd.get_dummies(df_train, columns=one_hot_encode_cols, dtype=int, drop_first=True)
df_test = pd.get_dummies(df_test, columns=one_hot_encode_cols, dtype=int, drop_first=True)

#-------------------------------------------------------------------------------------------------------------#

# Mapping func
def mapping(df_train, df_test, col_name, mapping_name):
    df_train[col_name] = df_train[col_name].map(mapping_name)
    df_test[col_name] = df_test[col_name].map(mapping_name)

# Dogum_Yeri, Ikametgah_Sehri, Lise_Sehir encode with mapping method.
iller_unique = il_ilce_list['il'].unique()
il_mapping = {il: idx for idx, il in enumerate(iller_unique)}

mapping(df_train, df_test, 'Dogum_Yeri', il_mapping)
mapping(df_train, df_test, 'Ikametgah_Sehri', il_mapping)
mapping(df_train, df_test, 'Lise_Sehir', il_mapping)

#-------------------------------------------------------------------------------------------------------------#

# Bölüm encode with mapping method.
missing_categories = {'Elektrik Elektronik Mühendisliği', 'İktisat (Ekonomi)', 'Hemşirelik (Fakülte)', 'işletme', 'iktisat', 'İşletme(İngilizce)'}

# Add category.
for category in missing_categories:
    df_train.loc[df_train["Bölüm"] == category, "Bölüm"] = category

departments_unique = df_train['Bölüm'].unique()
departments_unique = {category: idx for idx, category in enumerate(departments_unique)}
mapping(df_train, df_test, 'Bölüm', departments_unique)

#-------------------------------------------------------------------------------------------------------------#

# Universite_Kacinci_Sinif encode with mapping method.
üni_sınıf_mapping = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, 'Hazırlık': 7, 'Mezun': 8, 'Yüksek Lisans': 9}
mapping(df_train, df_test, 'Universite_Kacinci_Sinif', üni_sınıf_mapping)

#-------------------------------------------------------------------------------------------------------------#

# Baska_Kurumdan_Aldigi_Burs_Miktari encode with mapping method.
burs_miktar_mapping = {'0': 0, '0 - 499 ₺': 1, '500₺ - 999₺': 2, '1000₺ ve üstü': 3}
mapping(df_train, df_test, 'Baska_Kurumdan_Aldigi_Burs_Miktari', burs_miktar_mapping)

#-------------------------------------------------------------------------------------------------------------#

# Cinsiyet encode with mapping method.
cinsiyet_mapping = {'Erkek': 0, 'Kadın': 1}
mapping(df_train, df_test, 'Cinsiyet', cinsiyet_mapping)

#-------------------------------------------------------------------------------------------------------------#

def ordinal_enc(df_train, df_test, col_name, category_name):
    encoder = OrdinalEncoder(categories=category_name)
    df_train[col_name] = encoder.fit_transform(df_train[[col_name]])
    df_test[col_name] = encoder.transform(df_test[[col_name]])

# Choose Categories for Edu.
egitim_categories = [['Eğitimi yok', 'İlkokul', 'Ortaokul', 'Lise', 'Üniversite', 'Yüksek Lisans', 'Doktora']]

# Anne_Egitim_Durumu, Baba_Egitim_Durumu encode with ordinal encoder.
ordinal_enc(df_train, df_test, 'Anne_Egitim_Durumu', egitim_categories)
ordinal_enc(df_train, df_test, 'Baba_Egitim_Durumu', egitim_categories)

#-------------------------------------------------------------------------------------------------------------#

# Choose Categories for high school
lise_not_categories = [['25 - 49', '50 - 74', '75 - 100']]

# Lise_Mezuniyet_Notu encode with ordinal encoder.
ordinal_enc(df_train, df_test, 'Lise_Mezuniyet_Notu', lise_not_categories)

#-------------------------------------------------------------------------------------------------------------#

# Choose Categories for university
not_ort_categories = [['Hazırlığım', '0 - 1.79', '1.80 - 2.49', '2.50 - 2.99', '3.00 - 3.49', '3.50 - 4.00']]

# Universite_Not_Ortalamasi encode with ordinal encoder.
ordinal_enc(df_train, df_test, 'Universite_Not_Ortalamasi', not_ort_categories)

#-------------------------------------------------------------------------------------------------------------#

# Universite_Adi rearranged
df_train['Universite_Adi'] = df_train['Universite_Adi'].str.lower()
df_test['Universite_Adi'] = df_test['Universite_Adi'].str.lower()

en_iyi_20_üni = [
    
    'koç üniversitesi', 'hacettepe üniversitesi', 'orta doğu teknik üniversitesi', 'istanbul üniversitesi',
    'istanbul teknik üniversitesi', 'ankara üniversitesi', 'sabancı üniversitesi', 'gazi üniversitesi',
    'ege üniversitesi', 'ihsan doğramacı bilkent üniversitesi', 'istanbul üniversitesi cerrahpaşa',
    'gebze teknik üniversitesi', 'yıldız teknik üniversitesi', 'boğaziçi üniversitesi', 'marmara üniversitesi',
    'dokuz eylül üniversitesi', 'erciyes üniversitesi', 'atatürk üniversitesi', 'izmir teknoloji enstitüsü', 
    'fırat üniversitesi'
]

df_train['Universite_Adi'] = df_train['Universite_Adi'].apply(lambda x: x if str(x).lower() in en_iyi_20_üni else 'Diğer')
df_test['Universite_Adi'] = df_test['Universite_Adi'].apply(lambda x: x if str(x).lower() in en_iyi_20_üni else 'Diğer')

en_iyi_20_üni_sıralama = [['Diğer', 'fırat üniversitesi','izmir teknoloji enstitüsü','atatürk üniversitesi',
                          'erciyes üniversitesi','dokuz eylül üniversitesi','marmara üniversitesi',
                          'boğaziçi üniversitesi','yıldız teknik üniversitesi','gebze teknik üniversitesi',
                          'istanbul üniversitesi cerrahpaşa','ihsan doğramacı bilkent üniversitesi',
                          'ege üniversitesi','gazi üniversitesi','sabancı üniversitesi','ankara üniversitesi',
                          'istanbul teknik üniversitesi','istanbul üniversitesi','orta doğu teknik üniversitesi',
                          'hacettepe üniversitesi','koç üniversitesi']]

ordinal_enc(df_train, df_test, 'Universite_Adi', en_iyi_20_üni_sıralama)

#-------------------------------------------------------------------------------------------------------------#

# Lise_Adi rearranged.
df_train['Lise_Adi'] = df_train['Lise_Adi'].str.lower()
df_test['Lise_Adi'] = df_test['Lise_Adi'].str.lower()

en_iyi_20_lise = [
    'ankara fen lisesi','izmir fen lisesi','atatürk lisesi',
    'tofaş fen lisesi','adana fen lisesi','ankara pursaklar fen lisesi',
    'osman ulubaş kayseri fen lisesi','prof. dr. aziz sancar fen lisesi',
    'yusuf ziya öner fen lisesi','nilüfer borsa istanbul fen lisesi',
    'eyüp aygar fen lisesi','erbakır fen lisesi','aydın fen lisesi',
    'buca inci-özer tırnaklı fen lisesi','meram fen lisesi',
    'atatürk anadolu lisesi','samsun garip zeycan yıldırım fen lisesi',
    'sümer fen lisesi','kocaeli fen lisesi','vehbi dinçerler fen lisesi'
]

df_train['Lise_Adi'] = df_train['Lise_Adi'].apply(lambda x: x if str(x).lower() in en_iyi_20_lise else 'Diğer')
df_test['Lise_Adi'] = df_test['Lise_Adi'].apply(lambda x: x if str(x).lower() in en_iyi_20_lise else 'Diğer')

en_iyi_20_lise_siralama = [['Diğer','vehbi dinçerler fen lisesi','kocaeli fen lisesi','sümer fen lisesi',
                           'samsun garip zeycan yıldırım fen lisesi','atatürk anadolu lisesi',
                           'meram fen lisesi','buca inci-özer tırnaklı fen lisesi','aydın fen lisesi',
                           'erbakır fen lisesi','eyüp aygar fen lisesi','nilüfer borsa istanbul fen lisesi',
                           'yusuf ziya öner fen lisesi','prof. dr. aziz sancar fen lisesi',
                           'osman ulubaş kayseri fen lisesi','ankara pursaklar fen lisesi',
                           'adana fen lisesi','tofaş fen lisesi','atatürk lisesi','izmir fen lisesi','ankara fen lisesi']]

ordinal_enc(df_train, df_test, 'Lise_Adi', en_iyi_20_lise_siralama)

#-------------------------------------------------------------------------------------------------------------#

## Analyzing Correlations and Adjustments

In [ ]:
target = 'Degerlendirme_Puani'
df_ordered = pd.concat([df_train.drop(target,axis=1), df_train[target]],axis=1)
corr = df_ordered.corr(method='pearson')

# Create a mask so that we see the correlation values only once
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask,1)] = True

# Plot the heatmap correlation
plt.figure(figsize=(25,15), dpi=80)
sns.heatmap(corr, mask=mask, annot=True,fmt='.2f', linewidths=0.2)
plt.show()

In [ ]:
# High correlate ind variables droped for multicollinearity
df_train = df_train.drop(columns = ['Dogum_Yeri','Burs_Aliyor_mu__Hayır','Baska_Bir_Kurumdan_Burs_Aliyor_mu__Hayır'])
df_test = df_test.drop(columns = ['Dogum_Yeri','Burs_Aliyor_mu__Hayır','Baska_Bir_Kurumdan_Burs_Aliyor_mu__Hayır'])

In [ ]:
#bounds = {}
#for col in num_cols:
#    q1 = df_train[col].quantile(0.10)
#    q3 = df_train[col].quantile(0.90)
#    iqr = q3 - q1
#    lower_bound = q1 - 1.5 * iqr
#    upper_bound = q3 + 1.5 * iqr
#    bounds[col] = {'lower_bound': lower_bound, 'upper_bound': upper_bound}

# Her bir sayısal değişken için belirli aralık içindeki değerleri seçelim
#selected_values = {}
#for col in num_cols:
#    selected_values[col] = df[col][(df[col] >= bounds[col]['lower_bound']) & (df[col] <= bounds[col]['upper_bound'])]
    
# Belirli aralık dışındaki değerleri çıkaralım
#for col in num_cols:
#    df_train = df_train[(df_train[col] >= bounds[col]['lower_bound']) & (df_train[col] <= bounds[col]['upper_bound'])]

## Model Training


In [ ]:
# Model ve veri seti tanımları
X = df_train.drop(columns=['Degerlendirme_Puani'])
y = df_train['Degerlendirme_Puani']
X_test = df_test

model = cb.CatBoostRegressor(eval_metric='RMSE', loss_function='RMSE', silent=True, random_state=1453)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

mse_scores = []
rmse_scores = []
r2_scores = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mse_scores.append(mse)
    rmse = np.sqrt(mse)
    rmse_scores.append(rmse)
    r2 = r2_score(y_val, y_pred)
    r2_scores.append(r2)
    
print("MSE scores:", mse_scores)
print("Mean R^2 score:", np.mean(r2_scores))
print("Root Mean Square Error score:", np.mean(rmse_scores)) 

## Hyperparameter Optimization


In [ ]:
# Model and Dataset Definitions
X = df_train.drop(columns=['Degerlendirme_Puani'])
y = df_train['Degerlendirme_Puani']
X_test = df_test

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 3000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = cb.CatBoostRegressor(
            **params,
            eval_metric='RMSE',
            loss_function='RMSE',
            silent=True,
            random_state=1453
        )

        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=0)
        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)

# Optimizasyon çalışmasını başlat
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

In [ ]:
# Making Predictions
y_pred = model.predict(X_test).astype(int)

## Saving the Results

In [ ]:
# Saving the Submission File
submission = pd.DataFrame({'id': df_test['id'], 'Degerlendirme Puani': y_pred})
submission['id'] = submission['id'].astype('int32')
submission.to_csv('cb_son.csv', index=False)
print("\nTahminler 'submission.csv' dosyasına kaydedildi.")